In [1]:
import pandas as pd
import sqlite3

# Load the CSV file
file_path =r"C:\Users\moroi\OneDrive\Desktop\Spacex.csv"
df = pd.read_csv(file_path)

# Display first few rows to understand the structure
df.head()


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [2]:
# Clean column names: replace spaces and special characters for SQL compatibility
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('__', '_')

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
df.to_sql('spacex', conn, index=False, if_exists='replace')

# Preview cleaned columns
df.columns.tolist()


['Date',
 'Time_UTC',
 'Booster_Version',
 'Launch_Site',
 'Payload',
 'PAYLOAD_MASS_KG_',
 'Orbit',
 'Customer',
 'Mission_Outcome',
 'Landing_Outcome']

Task 1
Display the names of the unique launch sites in the space mission

In [4]:
query = "SELECT DISTINCT Launch_Site FROM spacex;"
unique_launch_sites = pd.read_sql_query(query, conn)


unique_launch_sites


,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


Task 2
Display 5 records where launch sites begin with the string 'CCA'

In [7]:
query = """
SELECT * FROM spacex
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;
"""

# Run the query
result = pd.read_sql_query(query, conn)

# Show the results
print(result)

         Date  Time_UTC Booster_Version  Launch_Site  \
0  2010-06-04  18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08  15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22   7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08   0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01  15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS_KG_  \
0               Dragon Spacecraft Qualification Unit                 0   
1  Dragon demo flight C1, two CubeSats, barrel of...                 0   
2                              Dragon demo flight C2               525   
3                                       SpaceX CRS-1               500   
4                                       SpaceX CRS-2               677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  
2  LEO (ISS

Task 3
Display the total payload mass carried by boosters launched by NASA (CRS)

In [12]:
query = """
SELECT SUM(PAYLOAD_MASS_KG_) AS Total_Payload_Mass
FROM spacex
WHERE Customer = 'NASA (CRS)';
"""

total_mass_crs = pd.read_sql_query(query, conn)
total_mass_crs

,Total_Payload_Mass
0,45596


Task 4
Display average payload mass carried by booster version F9 v1.1

In [13]:
query = """
SELECT AVG(PAYLOAD_MASS_KG_) AS Average_Payload_Mass
FROM spacex
WHERE Booster_Version = 'F9 v1.1';
"""

avg_mass_f9v1_1 = pd.read_sql_query(query, conn)
avg_mass_f9v1_1

,Average_Payload_Mass
0,2928.4


Task 5
List the date when the first succesful landing outcome in ground pad was acheived.
Hint:Use min function

In [15]:
query = """
SELECT MIN(Date) AS First_Ground_Pad_Landing_Date
FROM spacex
WHERE Landing_Outcome = 'Success (ground pad)';
"""

first_ground_landing = pd.read_sql_query(query, conn)
first_ground_landing

,First_Ground_Pad_Landing_Date
0,2015-12-22


Task 6
List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [16]:
query = """
SELECT Booster_Version
FROM spacex
WHERE Landing_Outcome = 'Success (drone ship)'
AND PAYLOAD_MASS_KG_ > 4000 
AND PAYLOAD_MASS_KG_ < 6000;
"""

drone_ship_boosters = pd.read_sql_query(query, conn)
drone_ship_boosters

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


Task 7
List the total number of successful and failure mission outcomes

In [17]:
query = """
SELECT Mission_Outcome, COUNT(*) as Count
FROM spacex
GROUP BY Mission_Outcome;
"""

mission_outcomes = pd.read_sql_query(query, conn)
mission_outcomes

,Mission_Outcome,Count
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


Task 8
List all the booster_versions that have carried the maximum payload mass. Use a subquery.

In [21]:
query = """
SELECT  Booster_Version , PAYLOAD_MASS_KG_
FROM spacex
WHERE PAYLOAD_MASS_KG_ = (SELECT MAX(PAYLOAD_MASS_KG_) FROM spacex);
"""

max_payload_boosters = pd.read_sql_query(query, conn)
max_payload_boosters

,Booster_Version,PAYLOAD_MASS_KG_
0,F9 B5 B1048.4,15600
1,F9 B5 B1049.4,15600
2,F9 B5 B1051.3,15600
3,F9 B5 B1056.4,15600
4,F9 B5 B1048.5,15600
5,F9 B5 B1051.4,15600
6,F9 B5 B1049.5,15600
7,F9 B5 B1060.2,15600
8,F9 B5 B1058.3,15600
9,F9 B5 B1051.6,15600


Task 9
List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
Note: SQLLite does not support monthnames. So you need to use substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.

In [22]:
query = """
SELECT 
	substr(Date, 6, 2) as Month,
	Booster_Version,
	Launch_Site,
	Landing_Outcome
FROM spacex
WHERE substr(Date, 0, 5) = '2015'
AND Landing_Outcome LIKE '%Failure%drone ship%';
"""

failed_drone_landings_2015 = pd.read_sql_query(query, conn)
failed_drone_landings_2015

,Month,Booster_Version,Launch_Site,Landing_Outcome
0,01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
1,04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


Task 10
Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [23]:
query = """
SELECT 
	Landing_Outcome, 
	COUNT(*) as Count
FROM 
	spacex
WHERE 
	Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY 
	Landing_Outcome
ORDER BY 
	Count DESC;
"""

landing_outcome_ranks = pd.read_sql_query(query, conn)
landing_outcome_ranks

,Landing_Outcome,Count
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1
